In [1]:
import requests, csv 
import us
import pandas as pd
import time
from easymoney.money import EasyPeasy
import json

/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# import the json with census variables
vars_url = "https://api.census.gov/data/2022/acs/acs5/subject/variables.json"
vars = requests.get(vars_url).json()

# turn the json into a pandas dataframe
vars_df = pd.DataFrame(vars['variables']).T
vars_df = vars_df.reset_index()
vars_df

,index,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
0,for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
1,in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
2,ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
3,S0804_C04_068E,Estimate!!Public transportation (excluding tax...,Means of Transportation to Work by Selected Ch...,float,S0804,0,NaN,NaN,"S0804_C04_068EA,S0804_C04_068M,S0804_C04_068MA",NaN
4,S0503_C02_078E,Estimate!!Foreign born; Born in Europe!!Civili...,Selected Characteristics of the Foreign-Born P...,float,S0503,0,NaN,NaN,"S0503_C02_078EA,S0503_C02_078M,S0503_C02_078MA",NaN
...,...,...,...,...,...,...,...,...,...,...
18822,S2402_C02_035E,"Estimate!!Male!!Full-time, year-round civilian...","Occupation by Sex for the Full-Time, Year-Roun...",int,S2402,0,NaN,NaN,"S2402_C02_035EA,S2402_C02_035M,S2402_C02_035MA",NaN
18823,S1002_C04_004E,Estimate!!60 years and over!!Percent distribut...,Grandparents,float,S1002,0,NaN,NaN,"S1002_C04_004EA,S1002_C04_004M,S1002_C04_004MA",NaN
18824,S0601_C02_009E,Estimate!!Native; born in state of residence!!...,Selected Characteristics of the Total and Nati...,float,S0601,0,NaN,NaN,"S0601_C02_009EA,S0601_C02_009M,S0601_C02_009MA",NaN
18825,S2411_C01_012E,Estimate!!Median earnings (dollars)!!Civilian ...,Occupation by Sex and Median Earnings in the P...,int,S2411,0,NaN,NaN,"S2411_C01_012EA,S2411_C01_012M,S2411_C01_012MA",NaN


In [3]:
vars_df = vars_df[['index', 'label']]
vars_df

,index,label
0,for,Census API FIPS 'for' clause
1,in,Census API FIPS 'in' clause
2,ucgid,Uniform Census Geography Identifier clause
3,S0804_C04_068E,Estimate!!Public transportation (excluding tax...
4,S0503_C02_078E,Estimate!!Foreign born; Born in Europe!!Civili...
...,...,...
18822,S2402_C02_035E,"Estimate!!Male!!Full-time, year-round civilian..."
18823,S1002_C04_004E,Estimate!!60 years and over!!Percent distribut...
18824,S0601_C02_009E,Estimate!!Native; born in state of residence!!...
18825,S2411_C01_012E,Estimate!!Median earnings (dollars)!!Civilian ...


In [4]:
# save as a csv
vars_df.to_csv("census_vars.csv", index=False)

In [6]:
# find the variables that contain the word "rent"
rent_vars = vars_df[vars_df['label'].str.contains('rent', case=False)]

In [8]:
# show all unique variables that contain the word "rent"
rent_vars['label'].unique()

array(['Estimate!!60 years and over!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house in the United States',
       'Estimate!!60 years and over!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house in the United States!!Same county',
       'Estimate!!60 years and over!!RESIDENCE 1 YEAR AGO!!Population 1 year and over!!Different house in the United States!!Different county',
       ...,
       'Estimate!!60 years and over!!Percent distribution of grandparents responsible for grandchildren!!Grandparents living with own grandchildren under 18 years!!RACE AND HISPANIC OR LATINO ORIGIN!!One race',
       'Estimate!!60 years and over!!Percent distribution of grandparents responsible for grandchildren!!Grandparents living with own grandchildren under 18 years!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White',
       'Estimate!!60 years and over!!Percent distribution of grandparents responsible for grandchildren!!Grandparents living with own grandchildren un

In [12]:
api_key  = "9330018ae4a2bd20da22ced1281145d9405dc349"

In [21]:
dataset = 'acs/acs5' # American Community Survey 5-year data
series = "B21004_001E" # Median Income


responses = [] 
for year in list(range(2010, 2020))+[2022]: # Loop through years, there's no data for 2020
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=county:*&in=state:*&key={api_key}'
    print(f'{year}: {data_url}')
    df = pd.read_json(data_url) # Read data from API into a pandas dataframe
    df.columns = df.iloc[0] # Set first row as column names
    df = df[1:] # Remove first row
    df["Year"] = year
    time.sleep(1) # Sleep for 1 second to avoid hitting the API too fast
    responses.append(df)

2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2012: https://api.census.gov/data/2012/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2013: https://api.census.gov/data/2013/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2014: https://api.census.gov/data/2014/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2015: https://api.census.gov/data/2015/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2016: https://api.census.gov/data/2016/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2017: https://api.census.gov/data/

In [17]:
url=f"https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key={api_key}"

response = requests.get(url)

d = response.json()

In [20]:
url

'https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349'

In [ ]:
https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=country:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349